### Load Materials Project API Key

In [1]:
import os, stat
from IPython.display import clear_output

try:
    user = str(input())
    clear_output()
    if not user.isalnum():
        raise TypeError('Wrong Key')
    if user == None:
        raise TypeError('Empty')
    with open(os.path.expanduser('~/.mpkey.txt'), 'w') as keyfile:
        keyfile.write(user)
    os.chmod(os.path.expanduser('~/.mpkey.txt'), stat.S_IREAD | stat.S_IWRITE)
    del user
    print("Success")
except:
    print("Something seems wrong with your key")

Success


## Libraries

This notebook requires several libraries to be imported. They are separated in blocks depending on their usage.

In [94]:
# These lines import both libraries and then define an array with elements to be used below

#Pymatgen and sub-libraries
import pymatgen
from pymatgen import MPRester, Element, Structure
from pymatgen.core import composition
from pymatgen.io.vasp import Vasprun
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.entries.compatibility import MaterialsProjectCompatibility
from pymatgen.util.plotting import *
from pymatgen.analysis.phase_diagram import *
#import pymatgen.core.composition
from pymatgen.apps.borg.hive import VaspToComputedEntryDrone
from pymatgen.entries.compatibility import MaterialsProjectCompatibility
import json

#Misc. Tools
import matplotlib.pyplot as plt
import random
import os
from itertools import combinations
import pandas as pd

#Import API Key
file = open(os.path.expanduser('~/.mpkey.txt'),"r+")
apikey = file.readline()
file.close()
rester = MPRester(apikey)

## 1. Query all selected materials  in MP database

In [241]:
#Query every structure in the MP database that has oxygen atom within compound. 
#Grab set of properties from database. Ti, Fe, Ni, W, C, Mo based.
# More elements "Si", "Zr", "V", "N", "B", "Ta", "Nb", "Hf", "Cr"
# We will have this setup as a data pipeline extraction procedure as iterations.
data = rester.query({"elements": "Cr",
                     "nelements": {"$gte": 2}},
                    ["task_id","pretty_formula","formula","volume","density","elements",
                     "e_above_hull","elasticity","unit_cell_formula"])

In [242]:
print('Number of oxide structures available on the MP database: %s' % len(data))
print('Example output: %s' % data[77])

Number of oxide structures available on the MP database: 5484
Example output: {'task_id': 'mp-1080200', 'pretty_formula': 'CrN2', 'formula': {'Cr': 1.0, 'N': 2.0}, 'volume': 52.24686162891472, 'density': 5.085813546639997, 'elements': ['Cr', 'N'], 'e_above_hull': 0.26577161920666015, 'elasticity': None, 'unit_cell_formula': {'Cr': 2.0, 'N': 4.0}}


## 2. Filtering data for energy stability
After data query has been completed we will begin filtering things just a bit. First we will strip any values that have an energy above the convex hull greater than 1 meV. 0 meV indicates most stable phase. This filter can be changed below.<br>


In [243]:
energy_cutoff_value = 1 #energy above convex hull cutoff [meV]

### 2.1 Filter data and calculate ionic packing fraction from compound query

In [244]:
#Create temporary list arrays for data calculation and manipulation. We will clear the array after each loop
atom_sample = []
atom_count = []
atom_radii = []
atom_volume = []
atom_volume_total = []

#Create modified dictionary to store values that we want from queried database of all oxides
property_dict_all = {}

for k in range(0,len(data)):
    #If Energy above convex Hull is larger than 1 meV we will skip that value and move on
    e_above_hull = data[k].get('e_above_hull')*1000
    if e_above_hull > energy_cutoff_value:
        continue
    #Grab values from data query. Data stored as dictionary so we call the 'keys' for 'values'
    mp_id = data[k].get('task_id')
    density = data[k].get('density')
    pretty_formula = data[k].get('pretty_formula')
    elastic_tensor = data[k].get('elasticity')
    composition = data[k].get('formula')
    oxide_volume = data[k].get('volume')
    atom_type_count = len(data[k].get('elements'))
    unit_cell_formula = data[k].get('unit_cell_formula')

    #Pull atom count for each atom type. Will be used to find summed atomic volumes and summed masses
    for type in np.arange(0,atom_type_count):
        atom_sample.append(data[k].get('elements')[type])     
        atom_count.append(data[k].get('unit_cell_formula').get(data[k].get('elements')[type]))
        
    #Do a quick query of each element type for ionic radius and atomic mass from Pymatgen
    for item in atom_sample:
        element_object = pymatgen.Element(item)
        radii_values = element_object.average_ionic_radius
        atom_radii.append(radii_values)

    #For each atom radii queried calculate the atomic volume
    for i in atom_radii:
        v = (4/3)*np.pi*i**3
        atom_volume.append(v)
        
    #Calculate the total atomic volume for each element type (E.g. 3*(Volume O atom))
    for i in np.arange(0,len(atom_count)):
        v_total = atom_count[i]*atom_volume[i]
        atom_volume_total.append(v_total)
        
    #Sum the volumes of all atoms in the oxide
    summed_volumes = sum(atom_volume_total)
    
    #Calculate packing fraction. Append values to dictionary with the Materials Project ID as key value
    packing_frac = summed_volumes/oxide_volume
    property_dict_all[str(mp_id)] = pretty_formula,composition,elastic_tensor,packing_frac,density,e_above_hull
    
    #Clear values for next loop
    atom_sample.clear()
    atom_count.clear()
    atom_radii.clear()
    atom_volume.clear()
    atom_volume_total.clear()

### 2.2 Convert our dictionary to a more pleasant looking dataframe. Pull the values from each key, separate them, and shove everything into a dataframe.

In [245]:
#Create blank lists for appending values
mp_id = []
pretty_formula = []
composition = []
ionic_packing_frac = []
density = []
elastic_tensor = []
e_above_hull = []
#Loop through all keys and extract their values. Append values into individual list arrays
for key in property_dict_all.keys():
    mp_id.append(key)
    values = property_dict_all.get(key)
    pretty_formula.append(values[0])
    composition.append(values[1])
    elastic_tensor.append(values[2])
    ionic_packing_frac.append(values[3])
    density.append(values[4])
    e_above_hull.append(values[5])    
#Stack arrays into a Numpy array. From here will append into a dataframe. Display dataframe below.
df = np.column_stack((mp_id,pretty_formula,composition,ionic_packing_frac,density,elastic_tensor,e_above_hull))
df_oxide = pd.DataFrame(df,columns=["mp_id","Formula","Composition","IPF","Density","Elastic_Tensor","E_above_Hull"])
display(df_oxide)

,mp_id,Formula,Composition,IPF,Density,Elastic_Tensor,E_above_Hull
0,mp-1007923,CrNi3,"{'Cr': 1.0, 'Ni': 3.0}",0.195062,8.61894,"{'G_Reuss': 97.0, 'G_VRH': 105.0, 'G_Voigt': 1...",0.26945
1,mp-1070544,CrGa4,"{'Cr': 1.0, 'Ga': 4.0}",0.118682,6.0189,None,0
2,mp-1078278,CrB4,"{'Cr': 1.0, 'B': 4.0}",0.125382,4.27914,None,0
3,mp-1080664,CrB,"{'Cr': 1.0, 'B': 1.0}",0.226432,6.26764,None,0
4,mp-1080757,Ta2CrB2,"{'Ta': 2.0, 'Cr': 1.0, 'B': 2.0}",0.151834,12.6566,None,0
...,...,...,...,...,...,...,...
654,mp-744625,ZnCrH15N5Cl4F,"{'Zn': 1.0, 'Cr': 1.0, 'H': 15.0, 'N': 5.0, 'C...",0.0650236,1.86923,None,0
655,mp-756910,K2CrPCO7,"{'K': 2.0, 'Cr': 1.0, 'P': 1.0, 'C': 1.0, 'O':...",0.507673,2.6028,None,0
656,mp-769568,Mn2V3Cr(PO4)6,"{'Mn': 2.0, 'V': 3.0, 'Cr': 1.0, 'P': 6.0, 'O'...",0.478665,3.24065,None,0
657,mp-776286,Ti3CrSn2(PO4)6,"{'Ti': 3.0, 'Cr': 1.0, 'Sn': 2.0, 'P': 6.0, 'O...",0.456562,3.43538,None,0


In [246]:
df_oxide.dtypes

mp_id             object
Formula           object
Composition       object
IPF               object
Density           object
Elastic_Tensor    object
E_above_Hull      object
dtype: object

In [247]:
df_oxide['Elastic_Tensor']

0      {'G_Reuss': 97.0, 'G_VRH': 105.0, 'G_Voigt': 1...
1                                                   None
2                                                   None
3                                                   None
4                                                   None
                             ...                        
654                                                 None
655                                                 None
656                                                 None
657                                                 None
658                                                 None
Name: Elastic_Tensor, Length: 659, dtype: object

In [248]:
df_oxide=df_oxide.dropna()

In [249]:
df_oxide

,mp_id,Formula,Composition,IPF,Density,Elastic_Tensor,E_above_Hull
0,mp-1007923,CrNi3,"{'Cr': 1.0, 'Ni': 3.0}",0.195062,8.61894,"{'G_Reuss': 97.0, 'G_VRH': 105.0, 'G_Voigt': 1...",0.26945
12,mp-12083,CrIr3,"{'Cr': 1.0, 'Ir': 3.0}",0.162656,18.6484,"{'G_Reuss': 208.0, 'G_VRH': 214.0, 'G_Voigt': ...",0
19,mp-12585,CrRh3,"{'Cr': 1.0, 'Rh': 3.0}",0.15708,10.8455,"{'G_Reuss': 146.0, 'G_VRH': 151.0, 'G_Voigt': ...",0
21,mp-1425,TiCr2,"{'Ti': 1.0, 'Cr': 2.0}",0.236119,6.23741,"{'G_Reuss': 75.0, 'G_VRH': 75.0, 'G_Voigt': 75...",0.0768329
23,mp-15617,Cr5B3,"{'Cr': 5.0, 'B': 3.0}",0.247747,6.58737,"{'G_Reuss': 195.0, 'G_VRH': 196.0, 'G_Voigt': ...",0
...,...,...,...,...,...,...,...
538,mp-554252,K2NaCrF6,"{'K': 2.0, 'Na': 1.0, 'Cr': 1.0, 'F': 6.0}",0.321992,2.961,"{'G_Reuss': 35.0, 'G_VRH': 35.0, 'G_Voigt': 35...",0
557,mp-1024999,HoCr2Si2C,"{'Ho': 1.0, 'Cr': 2.0, 'Si': 2.0, 'C': 1.0}",0.160212,6.83768,"{'G_Reuss': 110.0, 'G_VRH': 111.0, 'G_Voigt': ...",0
585,mp-31624,Sr2TaCrO6,"{'Sr': 2.0, 'Ta': 1.0, 'Cr': 1.0, 'O': 6.0}",0.589968,6.5567,"{'G_Reuss': 107.0, 'G_VRH': 107.0, 'G_Voigt': ...",0
587,mp-556264,KRb2CrF6,"{'K': 1.0, 'Rb': 2.0, 'Cr': 1.0, 'F': 6.0}",0.358013,3.42239,"{'G_Reuss': 25.0, 'G_VRH': 27.0, 'G_Voigt': 28...",0.343295


### 2.3 Observe the reduction from all oxide species to the limited set for stable structures. Plot the data below.

In [250]:
print('Number of structures available on the MP database: %s' % len(data))
print('Number of structures below %.1f meV: %s' %(energy_cutoff_value, len(df_oxide)))

Number of structures available on the MP database: 5484
Number of structures below 1.0 meV: 94


In [251]:
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot

plotly.offline.init_notebook_mode(connected=True)

#Layout design. Can change title, font, x/y-axis etc. Commented out pieces are height and width. 
#For plot in browser I prefer long horizontal plots. For presentations square images may be preferred. 
#Image can be directly saved by hovering over image and clicking camera icon. 
layout_IPF = go.Layout(title= "Oxide Query IPF vs. Density", hovermode= 'closest',
                   font = dict(family='Times New Roman',size=18),
                   xaxis= dict(title= '$Density \: [g/cm^{3}]$',zeroline= False, gridwidth= 2),
                   yaxis= dict(title= '$Ionic \: Packing \: Fraction \: [IPF]$',zeroline= False, gridwidth= 2),
                   height = 600,
                   width = 1200,
     showlegend= True                
)

#Scatter plot of collected data. Use df_oxide_all dataframe. df_oxide_all.Density will pull density column.
trace_all = go.Scatter(x = df_oxide.Density, y = df_oxide.IPF, mode = 'markers',
                    marker=dict(size=12, color='black'), text = df_oxide.Formula, name = 'All Queries')

trace_Al2O3 = go.Scatter(x = df_oxide[(df_oxide.Formula == 'Al2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                         y = df_oxide[(df_oxide.Formula == 'Al2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                         mode = 'markers', marker=dict(size=18, color='green'), text = 'Al2O3', name = 'Al2O3')
trace_Cr2O3 = go.Scatter(x = df_oxide[(df_oxide.Formula == 'Cr2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                         y = df_oxide[(df_oxide.Formula == 'Cr2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                         mode = 'markers', marker=dict(size=18, color='blue'), text = 'Cr2O3', name = 'Cr2O3')
trace_Y2O3 = go.Scatter(x = df_oxide[(df_oxide.Formula == 'Y2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                        y = df_oxide[(df_oxide.Formula == 'Y2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                        mode = 'markers', marker=dict(size=18, color='orange'), text = 'Y2O3', name = 'Y2O3')
trace_SiO2 = go.Scatter(x = df_oxide[(df_oxide.Formula == 'SiO2') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                        y = df_oxide[(df_oxide.Formula == 'SiO2') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                        mode = 'markers', marker=dict(size=18, color='magenta'), text = 'SiO2', name = 'SiO2')

data_IPF = [trace_all,trace_Al2O3,trace_Cr2O3,trace_Y2O3,trace_SiO2]#,trace0,trace2]
fig_IPF = go.Figure(data_IPF, layout=layout_IPF)
iplot(fig_IPF)

## 3. Filtering data for elasticity

### 3.1 Observe the reduction from all oxide species to the limited set for stable structures, and ones with mechanical property data


In [252]:
print('Number of structures available on the MP database: %s' % len(data))
print('Number of structures below %.1f meV and with elasticity data: %s' %(energy_cutoff_value, len(df_oxide)))

Number of structures available on the MP database: 5484
Number of structures below 1.0 meV and with elasticity data: 94


### 3.2 Pull dataframe column with dictionary of elastic constants. Create separate columns for each value instead of inset in dictionary.

In [253]:
#Pull dataframe column with dictionary of elastic constants
elastics = df_oxide.Elastic_Tensor
G_VRH = []
K_VRH = []
Elastic_Anisotropy = []
poisson_ratio = []
Y_Modulus = []

for k in elastics.index: #len(df_oxide))
    G_VRH.append(elastics[k].get('G_VRH'))
    K_VRH.append(elastics[k].get('K_VRH'))

    Elastic_Anisotropy.append(elastics[k].get('elastic_anisotropy'))
    poisson_ratio.append(elastics[k].get('poisson_ratio'))
    Y_Modulus.append(2*elastics[k].get('G_VRH')*(1+elastics[k].get('poisson_ratio')))
    
elasticity_data = np.column_stack((G_VRH, K_VRH, Elastic_Anisotropy, poisson_ratio, Y_Modulus))

df_elasticity = pd.DataFrame(elasticity_data,columns=["G_VRH", 
                                                      "K_VRH", 
                                                      "Elastic_Anisotropy", 
                                                      "poisson_ratio",
                                                     "Y_Modulus"])



df_oxide = df_oxide.reset_index(drop=True)
df_elasticity = df_elasticity.reset_index(drop=True)
df_oxide = pd.concat([df_oxide,df_elasticity],axis=1)
display(df_oxide)

#Remove erroneous poisson ratios
void_test = []
row_delete = []
for k in range(0,len(df_elasticity)):
    if df_oxide['poisson_ratio'].iloc[k] < 0:
        row_delete.append(k)
        

index = df_oxide.index[row_delete]
df_oxide.drop(index,axis=0,inplace=True)

,mp_id,Formula,Composition,IPF,Density,Elastic_Tensor,E_above_Hull,G_VRH,K_VRH,Elastic_Anisotropy,poisson_ratio,Y_Modulus
0,mp-1007923,CrNi3,"{'Cr': 1.0, 'Ni': 3.0}",0.195062,8.61894,"{'G_Reuss': 97.0, 'G_VRH': 105.0, 'G_Voigt': 1...",0.26945,105.0,222.0,0.82,0.29,270.90
1,mp-12083,CrIr3,"{'Cr': 1.0, 'Ir': 3.0}",0.162656,18.6484,"{'G_Reuss': 208.0, 'G_VRH': 214.0, 'G_Voigt': ...",0,214.0,307.0,0.28,0.22,522.16
2,mp-12585,CrRh3,"{'Cr': 1.0, 'Rh': 3.0}",0.15708,10.8455,"{'G_Reuss': 146.0, 'G_VRH': 151.0, 'G_Voigt': ...",0,151.0,230.0,0.38,0.23,371.46
3,mp-1425,TiCr2,"{'Ti': 1.0, 'Cr': 2.0}",0.236119,6.23741,"{'G_Reuss': 75.0, 'G_VRH': 75.0, 'G_Voigt': 75...",0.0768329,75.0,199.0,0.03,0.33,199.50
4,mp-15617,Cr5B3,"{'Cr': 5.0, 'B': 3.0}",0.247747,6.58737,"{'G_Reuss': 195.0, 'G_VRH': 196.0, 'G_Voigt': ...",0,196.0,291.0,0.03,0.23,482.16
...,...,...,...,...,...,...,...,...,...,...,...,...
89,mp-554252,K2NaCrF6,"{'K': 2.0, 'Na': 1.0, 'Cr': 1.0, 'F': 6.0}",0.321992,2.961,"{'G_Reuss': 35.0, 'G_VRH': 35.0, 'G_Voigt': 35...",0,35.0,79.0,0.00,0.31,91.70
90,mp-1024999,HoCr2Si2C,"{'Ho': 1.0, 'Cr': 2.0, 'Si': 2.0, 'C': 1.0}",0.160212,6.83768,"{'G_Reuss': 110.0, 'G_VRH': 111.0, 'G_Voigt': ...",0,111.0,160.0,0.16,0.22,270.84
91,mp-31624,Sr2TaCrO6,"{'Sr': 2.0, 'Ta': 1.0, 'Cr': 1.0, 'O': 6.0}",0.589968,6.5567,"{'G_Reuss': 107.0, 'G_VRH': 107.0, 'G_Voigt': ...",0,107.0,187.0,0.02,0.26,269.64
92,mp-556264,KRb2CrF6,"{'K': 1.0, 'Rb': 2.0, 'Cr': 1.0, 'F': 6.0}",0.358013,3.42239,"{'G_Reuss': 25.0, 'G_VRH': 27.0, 'G_Voigt': 28...",0.343295,27.0,36.0,0.48,0.21,65.34


In [254]:
# export all the dataframes for all seven datasets.
df_oxide.to_csv (r'C:\Users\Kayla Yano\Class\Trial\Cr_coumpounds_dataframe.csv', index = True, header=True)


### 3.3.1 Plot material properties from dataframe

In [255]:
#Layout design. Can change title, font, x/y-axis etc. Commented out pieces are height and width. For plot in browser I prefer long horizontal plots. For presentations square images may be preferred. 
#Image can be directly saved by hovering over image and clicking camera icon. 
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot

layout_IPF = go.Layout(title= "Ti_based Query IPF vs. Density", hovermode= 'closest',
                   font = dict(family='Times New Roman',size=18),
                   xaxis= dict(title= '$Density \: [g/cm^{3}]$',zeroline= False, gridwidth= 2),
                   yaxis= dict(title= '$Ionic Packing Fraction \: [IPF]$',zeroline= False, gridwidth= 2),
                   height = 600,
                   width = 1200,
     showlegend= True                
)

#Scatter plot of collected data. Use df_oxide_all dataframe. df_oxide_all.Density will pull density column. Could change Density or IPF to Molar Volume or simply volume if you wanted.
trace_all = go.Scatter(x = df_oxide.Density, y = df_oxide.IPF, mode = 'markers',
                    marker=dict(size=12, color='black'), text = df_oxide.Formula, name = 'All Queries')

trace_TiC = go.Scatter(x = df_oxide[(df_oxide.Formula == 'TiC') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                         y = df_oxide[(df_oxide.Formula == 'TiC') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                         mode = 'markers', marker=dict(size=18, color='green'), text = 'TiC', name = 'TiC')
trace_TiN = go.Scatter(x = df_oxide[(df_oxide.Formula == 'TiN') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                         y = df_oxide[(df_oxide.Formula == 'TiN') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                         mode = 'markers', marker=dict(size=18, color='blue'), text = 'TiN', name = 'TiN')
trace_TiB = go.Scatter(x = df_oxide[(df_oxide.Formula == 'TiB') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                        y = df_oxide[(df_oxide.Formula == 'TiB') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                        mode = 'markers', marker=dict(size=18, color='orange'), text = 'TiB', name = 'TiB')
trace_TiO2 = go.Scatter(x = df_oxide[(df_oxide.Formula == 'TiO2') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                        y = df_oxide[(df_oxide.Formula == 'TiO2') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                        mode = 'markers', marker=dict(size=18, color='magenta'), text = 'TiO2', name = 'TiO2')

data_IPF = [trace_all,trace_TiC,trace_TiN,trace_TiB,trace_TiO2]
fig_IPF = go.Figure(data_IPF, layout=layout_IPF)
iplot(fig_IPF)

### 3.3.2 Plot material properties from dataframe using 3rd property as color key

In [256]:
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot
layout_IPF_shear_modulus = go.Layout(title= "C-based Compounds Bulk Modulus vs. Shear Modulus", hovermode= 'closest',
                   font = dict(family='Times New Roman',size=18),
                   xaxis= dict(title= '$Shear \: Modulus \: [GPa]$',zeroline= False, gridwidth= 2),
                   yaxis= dict(title= '$Bulk \: Modulus [GPa]$',zeroline= False, gridwidth= 2),
                   height = 600,
                   width = 1200,
     showlegend= False              
)

#Scatter plot of collected data. Use df_oxide_all dataframe. df_oxide_all.Density will pull density column. Could change Density or IPF to Molar Volume or simply volume if you wanted.
trace_all_shear = go.Scatter(x = abs(df_oxide.G_VRH), y = df_oxide.K_VRH, mode = 'markers', text = df_oxide.Formula,
                            marker=dict(size=12, color=(df_oxide['poisson_ratio']),
                                colorbar = dict(title={ 'text': "Poisson Ratio", 
                                                     'font': {'family':'Georgia', 'size': 18}} , 
                                              tickfont={'family':'Georgia', 'size': 16 })))#, name = 'All Queries'))

                             
data_IPF_shear = [trace_all_shear]
fig_IPF_shear = go.Figure(data_IPF_shear, layout=layout_IPF_shear_modulus)
iplot(fig_IPF_shear)

In [259]:
layout_test = go.Layout(title= "Cr compounds", hovermode= 'closest',
                  font = dict(family='Times New Roman',size=36),
                  xaxis= dict(title= 'Density [g/cm<sup>-3</sup>]',zeroline= False, gridwidth= 2), #To change range put command inside xaxis dictionary after gridwidth range=[-XX,XX]
                  yaxis= dict(title= "Young's Modulus [GPa]",zeroline= False, gridwidth= 2,range=[0,600]), #To change range put command inside yaxis dictionary after gridwidth range=[-YY,YY]
                  height = 800,
                  width = 800,
    showlegend= False              
)

#Scatter plot of collected data. Use df_oxide_all dataframe. df_oxide_all.Density will pull density column. Could change Density or IPF to Molar Volume or simply volume if you wanted.
trace_test = go.Scatter(x = df_oxide.Density, y = df_oxide.Y_Modulus, mode = 'markers', text = df_oxide.Formula,
                           marker=dict(size=12, color=(df_oxide['poisson_ratio']),
                               colorbar = dict(title={ 'text': "Poisson Ratio", 
                                                    'font': {'family':'Georgia', 'size': 18}},  
                                             tickfont={'family':'Georgia', 'size': 16 })), )#, name = 'All Queries'))


import plotly.express as px
data_test = [trace_test]
fig_test = go.Figure(data_test, layout=layout_test)
# fig_test.update_layout(color_continuous_scale=px.colors.sequential.Viridis)
iplot(fig_test)